In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:28<00:00, 43.1MB/s]
100% 1.06G/1.06G [00:28<00:00, 40.5MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [7]:
code_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3)
)

58889256/58889256 [==============================] - 2s 0us/step


In [8]:
code_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [9]:
model = Sequential()

model.add(code_base)

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [11]:
code_base.trainable = False

In [16]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (150,150)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (150,150)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [17]:
def process(image,label):
    image = tensorflow.cast(image/255,tensorflow.float32)
    return image,label

    train_ds = train_ds.map(process)
    validation_ds = validation_ds.map(process)


In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 67s 88ms/step - loss: 0.4844 - accuracy: 0.9352 - val_loss: 0.1377 - val_accuracy: 0.9472
Epoch 2/10
625/625 [==============================] - 53s 85ms/step - loss: 0.0813 - accuracy: 0.9712 - val_loss: 0.1658 - val_accuracy: 0.9460
Epoch 3/10
625/625 [==============================] - 53s 84ms/step - loss: 0.0521 - accuracy: 0.9822 - val_loss: 0.1977 - val_accuracy: 0.9502
Epoch 4/10
625/625 [==============================] - 53s 85ms/step - loss: 0.0380 - accuracy: 0.9878 - val_loss: 0.2445 - val_accuracy: 0.9466
Epoch 5/10
625/625 [==============================] - 53s 84ms/step - loss: 0.0403 - accuracy: 0.9864 - val_loss: 0.2592 - val_accuracy: 0.9512
Epoch 6/10
625/625 [==============================] - 53s 84ms/step - loss: 0.0344 - accuracy: 0.9893 - val_loss: 0.3507 - val_accuracy: 0.9486
Epoch 7/10
625/625 [==============================] - 53s 85ms/step - loss: 0.0296 - accuracy: 0.9916 - val_loss: 0.3032 - val_accuracy: